## How To Build Trainable Feature Maps from the Qiskit Circuit Library

In this guide, we will show how to build trainable feature maps from existing circuits in the Qiskit circuit library. Each approach will involve reassigning some parameters originally reserved for input data to instead be trainable parameters.

To build a trainable feature map, we require the following:

1. A circuit containing parameterized gates
2. A partition of circuit parameters into two sets: input parameters (encode the data) and user (trainable) parameters
3. After partitioning parameters, the dimensionality of the input data must equal the number of input parameters

### Option 1: Partition the Parameters of a Single Circuit

The main distinction between a feature map and a _parameterized_ feature map is the presence of parameters not associated to our input data. In other words, a feature map contains input parameters (encoding the dataset), and a parameterized feature map contains both input parameters as well as user parameters (which are trainable). 

One way to generate a parameterized feature map from an existing Qiskit feature map is to reassign some of the input parameters to be user parameters instead. If you go down this path, take care to ensure that you retain enough input parameters to match the dimensionality of your data. 

In [1]:
#pylint: disable=protected-access
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit import ParameterVector

Let's start with a two-qubit feature map from the Qiskit circuit library. By default, this is not parametrized and contains two input parameters `x[0]` and `x[1]` encoding the components of each data sample.

In [2]:
# Define a (non-parameterized) feature map from the Qiskit circuit library
fm = ZZFeatureMap(2)
input_params = fm.parameters
fm.draw()

┌──────────────────────────┐
q_0: ┤0                         ├
     │  ZZFeatureMap(x[0],x[1]) │
q_1: ┤1                         ├
     └──────────────────────────┘

Let's partition the input parameters into two sets such that the second one is reassigned to be a user (trainable) parameter:

In [3]:
# split params into two disjoint sets
input_params = fm.parameters[::2]
user_params = fm.parameters[1::2]
print("input_params:", input_params)
print("user_params:", user_params)

input_params: [ParameterVectorElement(x[0])]
user_params: [ParameterVectorElement(x[1])]


For clarity, we will manually reassign the feature map parameters such that the new parameters are properly named. (Renaming is not strictly required in this example; however, in the example below it will be necessary to prevent name collisions.)

In [4]:
# define new parameter vectors for the input and user parameters
new_input_params = ParameterVector('x', len(input_params))
new_user_params = ParameterVector('θ', len(user_params))

# resassign the origin feature map parameters
param_reassignments = {}
for i, p in enumerate(input_params):
    param_reassignments[p] = new_input_params[i]
for i, p in enumerate(user_params):
    param_reassignments[p] = new_user_params[i]

fm.assign_parameters(param_reassignments, inplace=True)

input_params = new_input_params
user_params = new_user_params

print("input_params:", input_params)
print("user_params:", user_params)
fm.draw()

input_params: x, ['x[0]']
user_params: θ, ['θ[0]']


┌──────────────────────────┐
q_0: ┤0                         ├
     │  ZZFeatureMap(x[0],θ[0]) │
q_1: ┤1                         ├
     └──────────────────────────┘

### Option 2: Compose Multiple Circuits

We can build a parameterized feature map out of existing Qiskit library circuits by composing them to form a larger composite circuit. However, if two circuits have (different) parameters that share the same name, qiskit will not allow us to compose them. 

To resolve this issue, we will simply rename our user parameters to prevent name collisions. As a nice side effect, our parameter names will also be more accurate and helpful. Again, note that our parameter names are automatically updated in our feature map circuit.

*Note: although both options we show in this guide use two qubits, Option 2 results in a feature map that accepts two-dimensional data while Option 1 results in a feature map for one-dimensional data.*

In [5]:
# Define two circuits
circ1 = ZZFeatureMap(2)
circ2 = ZZFeatureMap(2)
input_params = circ1.parameters
user_params = ParameterVector('θ', 2)

# Reassign new parameters to circ2 so there are no name collisions
circ2.assign_parameters(user_params, inplace=True)

# Compose to build a parameterized feature map
fm = circ2.compose(circ1)
print("input_params:", list(input_params))
print("user_params:", user_params)
fm.draw()

input_params: [ParameterVectorElement(x[0]), ParameterVectorElement(x[1])]
user_params: θ, ['θ[0]', 'θ[1]']


┌──────────────────────────┐┌──────────────────────────┐
q_0: ┤0                         ├┤0                         ├
     │  ZZFeatureMap(θ[0],θ[1]) ││  ZZFeatureMap(x[0],x[1]) │
q_1: ┤1                         ├┤1                         ├
     └──────────────────────────┘└──────────────────────────┘

###  Option 3: Build from Scratch

As a user, you are free to define your own trainable feature map and we encourage you to do so! This is a growing area of research, and understanding what makes a good trainable feature map is an important next step toward realizing practical quantum machine learning. 

In our [guide on creating custom quantum feature maps](https://github.com/IBM-Quantum-prototypes/quantum-kernel-training/blob/main/docs/how_tos/create_custom_quantum_feature_map.ipynb) we give an example of a trainable feature map tailored to classifying datasets with a particular group structure. 

In [1]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright